In [94]:
import os
import re
import shutil
import sqlite3
from collections import defaultdict

import numpy as np
import pandas as pd
from Bio import Phylo, SeqIO, AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

META_FILE = "/Data2/Nextstrain_data/metadata-5.tsv"
MSA_FILE = "/Data2/Nextstrain_data/msa_0814.fasta"
SQLITE_FILE = "/Data2/Nextstrain_data/sars2.db"

TREES_DIR = "trees"
OUT_DIR = "renamed_trees"

if os.path.exists(OUT_DIR):
    shutil.rmtree(OUT_DIR)
os.mkdir(OUT_DIR)

In [ ]:
if os.path.exists(SQLITE_FILE):
    os.remove(SQLITE_FILE)

conn = sqlite3.connect(SQLITE_FILE)
cur = conn.cursor()

with conn:
    cur.execute('''
        CREATE TABLE records (
            accession TEXT PRIMARY KEY,
            sequence TEXT NOT NULL
        )
    ''')

with conn:
    for record in SeqIO.parse(MSA_FILE, "fasta"):
        (accession, ) = re.findall(r"EPI_ISL_[0-9]+", record.description)
        cur.execute(
            "INSERT INTO records VALUES (:accession, :sequence)",
            { "accession": accession, "sequence":  str(record.seq) }
        )

In [10]:
matched_files = defaultdict(list)
for fn in os.listdir(TREES_DIR):
    m = re.findall(r"[0-9]{4}-[0-9]{2}-[0-9]{2}", fn)
    if m:
        (date_time, ) = m
        matched_files[date_time].append(fn)

In [88]:
missed = set()

for date_time, (fn, fn2) in matched_files.items():
    if fn.endswith("nwk"):
        tree_fn = os.path.join(TREES_DIR, fn)
        meta_fn = os.path.join(TREES_DIR, fn2)
    else:
        tree_fn = os.path.join(TREES_DIR, fn2)
        meta_fn = os.path.join(TREES_DIR, fn)
    tree = Phylo.read(tree_fn, "newick")
    metadata = pd.read_csv(meta_fn, sep="\t", index_col=0, quoting=3)
    n_drop = 0
    out_seqs = []
    n_tips = len(tree.get_terminals())
    for tip in tree.get_terminals():
        accession = tip.name
        tip.name = metadata.loc[tip.name, "gisaid_epi_isl"]
        if tip.name is None:
            print(accession)
        cur.execute("SELECT sequence FROM records WHERE accession=?", (tip.name, ))
        res = cur.fetchone()
        if res is None:
            tree.prune(tip)
            missed.add(tip.name)
            n_drop += 1
        else:
            (sequence, ) = res
            out_seqs.append(SeqRecord(
                Seq(sequence),
                id=tip.name,
                description=""
            ))
    print(date_time, n_tips, n_drop)
    out_dir = os.path.join(OUT_DIR, date_time)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    for tip in tree.get_terminals():
        if tip.name is None:
            tree.prune(tip)
    out_seqs = MultipleSeqAlignment(out_seqs)
    AlignIO.write(out_seqs, os.path.join(out_dir, date_time + ".fasta"), "fasta")
    Phylo.write(tree, os.path.join(out_dir, date_time + ".nwk"), "newick")

2020-06-27 3087 128
2020-09-03 4734 179
2021-05-26 3796 215
2020-04-13 3592 143
2021-03-03 4011 289
2021-06-17 3997 284
2020-05-15 5368 229
2020-06-10 2645 88
2020-09-18 4640 152
2020-12-20 3510 178
2020-07-07 3322 143
2020-04-14 3823 146
2021-01-26 3918 227
2020-04-27 3650 152
2020-06-18 2872 96
2021-02-25 3992 244
2020-07-22 3868 143
2021-01-22 3926 240
2021-02-01 3838 240
2020-05-16 5430 232
2020-04-24 3567 144
2021-06-22 3799 256
2020-09-09 4417 161
2020-06-17 2857 92
2020-05-26 4307 169
2021-03-06 3960 258
2020-05-04 4533 184
2020-07-15 3644 170
2021-07-05 3968 272
2020-10-01 4992 201
2021-04-12 3969 279
2020-08-17 4514 155
2020-11-12 3375 162
2021-04-20 3900 275
2020-06-19 2893 89
2020-09-08 4403 156
2020-05-12 5113 227
2021-06-09 3991 291
2021-06-16 3873 261
2020-05-13 5158 232
2021-03-30 3955 239
2020-11-19 3333 154
2021-05-04 3903 247
2021-01-21 4046 212
2020-05-22 4484 198
2020-06-23 2986 100
2020-11-26 3413 166
2020-07-17 3746 137
2020-10-20 3645 198
2020-12-14 3456 160
2021

In [95]:
missed.discard(np.nan)

with open("missed.csv", "w") as f:
    f.write("\n".join(missed))
    f.write("\n")

In [98]:
len(missed)

10741